# Descision Tree
`05_descision_tree.ipynb`

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

wine = pd.read_csv('./wine.csv')
wine.head()
wine.info()
wine.describe()

In [ ]:
from sklearn.model_selection import train_test_split

# 데이터 분리
X = wine[['alcohol', 'sugar', 'pH']].to_numpy()
y = wine['class'].to_numpy()

# 훈련:테스트 = 8:2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape

In [ ]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train_scaled, y_train)

print('훈련점수: ', lr.score(X_train_scaled, y_train))
print('테스트점수: ', lr.score(X_test_scaled, y_test))

print(lr.classes_)
lr.predict_proba(X_test_scaled[:5])

print(lr.coef_, lr.intercept_)

In [ ]:
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train_scaled, y_train)

print('훈련점수: ', dt.score(X_train_scaled, y_train))
print('테스트점수: ', dt.score(X_test_scaled, y_test))

In [ ]:
from sklearn.tree import plot_tree

# 지니불순도 1 - (음성클래스비율^2 + 양성클래스비율^2)
gini = 1 - ( (1258/5197)**2 + (3939/5197)**2)
print(gini)

plt.figure(figsize=(12,10))
plot_tree(dt, max_depth=2, filled=True, feature_names=['alcohol', 'sugar', 'pH'])
plt.show()

In [ ]:
dt = DecisionTreeClassifier(max_depth=3, random_state=42)
dt.fit(X_train, y_train)
print(dt.feature_importances_)
print('훈련점수: ', dt.score(X_train, y_train))
print('테스트점수: ', dt.score(X_test, y_test))


plt.figure(figsize=(20, 15))
plot_tree(dt, filled=True, feature_names=['alcohol', 'sugar', 'pH'])
plt.show()

## 교차 검증 (Cross Validation)

테스트 셋은 모델 검증 마지막 단계에서 1번만 확인.

In [ ]:
# 훈련셋 (sub) | 검증셋 (val) | 테스트셋 (test)
# 80%                 20%
# 60%     20%         20%

In [ ]:
wine = pd.read_csv('./wine.csv')
X = wine[['alcohol', 'sugar', 'pH']]
y = wine['class']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# 이 부분은 뒤에 나올 cross_validate 를 수동으로 해본것! 실제로 우리가 실행하진 않을것
X_sub, X_val, y_sub, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [ ]:
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_sub, y_sub)
print('훈련: ', dt.score(X_sub, y_sub))
print('검증: ', dt.score(X_val, y_val))

In [ ]:
# K-Fold Cross Validation (데이터를 쪼개서 훈련-검증 데이터를 바꿔가며 검증 진행 -> 점수 평균)
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, X_train, y_train)
display(scores)
print(np.mean(scores['test_score']))

In [ ]:
# 폴드 기본값 5가 아닌 다른 값을 쓰고 싶을 경우
from sklearn.model_selection import StratifiedKFold
#                                       접기 전에 최초 1회 셔플링    
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, X_train, y_train, cv=splitter)
print(np.mean(scores['test_score']))

## 하이퍼 파라미터 튜닝
1. GridSearch 을 진행할 하이퍼 파라미터 선택
2. 그리드 서치 수행(fit)
3. 최적 조합을 찾고, `gs` 객체에 저장됨.
4. 그리드 서치는 최상의 매개변수에서, 전체 훈련 세트를 사용해 최종 모델을 훈련

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006],
}
                  
gs = GridSearchCV(
    DecisionTreeClassifier(random_state=42),  # 모델
    param_grid=params,  # 확인할 하이퍼파라미터의 이름: 값들
    n_jobs=-1,  # CPU 최대 코어
)

gs.fit(X_train, y_train)

# Grid Search 결과 가장 좋은 파라미터 조합으로 모델 만들기
print(gs.best_params_, gs.cv_results_['mean_test_score'])

dt = gs.best_estimator_
dt.score(X_train, y_train)

In [ ]:
params = {
    # 노드 분할을 위한 최소 불순도
    'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
    # 트리 깊이
    'max_depth': range(5, 20, 1),
    # 노드를 나누기 위한 최소 샘플 수
    'min_samples_split': range(2, 100, 10),
}

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(X_train, y_train)

print(gs.best_params_)
print(gs.best_score_)

In [ ]:
# 내부 교차검증 결과 가장 높은 평균 점수
print(np.max(gs.cv_results_['mean_test_score']))
print(gs.best_score_)
# 모든 하이퍼파라미터 교차검증 끝에 찾은 학습이 끝난 최고의 모델
dt_best = gs.best_estimator_
# 로 아껴놨던 테스트 진행 -> 점수
dt_best.score(X_test, y_test)


In [ ]:
from scipy.stats import uniform, randint

# 주어진 범위에서 고르게 값을 뽑는다.(randint->정수, uniform->실수)
rgen = randint(0, 10)
print(rgen.rvs(10))

print(np.unique(rgen.rvs(1000), return_counts=True))

ugen = uniform(0, 1)
print(ugen.rvs(10))

In [ ]:
params = {
    # 노드 분할을 위한 최소 불순도
    'min_impurity_decrease': uniform(0.0001, 0.001),
    # 트리 깊이
    'max_depth': randint(10, 50),
    # 노드를 나누기 위한 최소 샘플 수
    'min_samples_split': randint(2, 25),
    # 리프 노드 개수 최소값
    'min_samples_leaf': randint(1, 25),
}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(
    DecisionTreeClassifier(random_state=42),
    params,
    n_iter=1000,
    n_jobs=-1,
    random_state=42
)

gs.fit(X_train, y_train)
print(gs.best_params_)

In [ ]:
gs.best_score_
# 결정트리 모델 인스턴스
dt = gs.best_estimator_

print('최종 테스트 결과: ', dt.score(X_test, y_test))